# Radiative model intercomparison tool

This script is a tool to connect two assessments created to determine the ideal radiative transfer (RT) models to use for various applications. These assessments are stored as a matrix in .csv files, the contents of which can be altered by the user of this script. The two assessments are of:

1. Needs matrix: this identifies the various ways in which RT models could be used, and the requirements for each of those applications
2. Capability matrix: this identifies several RT models, and their capabilities for the requirements described in (1)

For a given RT model application, (1) and (2) are combined to provide scores for the success of each RT model. The results are plotted in a color-coded chart, and saved to a .csv file

history: 
created by kirk.knobelspiesse@nasa.gov, last updated 10/1/2020


## Libraries

In [1]:
import numpy as np
import pandas as pd
print('Using pandas version ',pd.__version__)
!pip install qgrid
import qgrid 
%matplotlib inline

Using pandas version  1.0.5


## Read Data

This section reads in CSV files with the radiative transfer "Needs" file and corresponding "Capabilities" file for each RT model. Note the path will need to be modified for your purposes.

In [2]:
rt_path='/Users/kknobels/Box_Sync/PACE/RT/'      #path is here
fname_capability=rt_path+'/RTmodelsPACE_cap.csv'
fname_needs=rt_path+'/RTmodelsPACE_need.csv'

cap = pd.read_csv(fname_capability,index_col=0)
need = pd.read_csv(fname_needs,index_col=0)
for capcol in cap.columns:       #convert columns to categorical
    cap[capcol] = cap[capcol].astype('category')    
for needcol in need.columns:       #convert columns to categorical
    need[needcol] = need[needcol].astype('category')    

## Examine "needs matrix"

The needs matrix describes the RT model requirements in one of three categories (yes, no, somewhat) for various usages, such as recreating heritage lookup tables. This table provides the means to alter those descriptions from the default values in the loaded csv files. Note the possibility to expand the table to fill the screen.

In [3]:
qgrid_widget = qgrid.show_grid(need, show_toolbar=True, grid_options={'forceFitColumns': True, 'column_fit_size_min':2})
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [4]:
need=qgrid_widget.get_changed_df()        #capture any adjustments that were made

## Examine "capabilties matrix"

The capabilities matrix describes the abilties of each RT model in one of three categories (yes, no, somewhat) for various metrics, such as abiltiy to calculate 3D scattering. This table provides the means to alter those descriptions from the default values in the loaded csv files. Note the possibility to expand the table to fill the screen.

In [5]:
qgrid_widget = qgrid.show_grid(cap, show_toolbar=True, grid_options={'forceFitColumns': True, 'column_fit_size_min':2})
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [6]:
cap=qgrid_widget.get_changed_df()        #capture any adjustments that were made

## Combine needs and capabilities matricies

Combine needs matrix and capabilities matrix into an overall dataframe describing how the capabilities of each RT model fit different ways of using RT models. Scoring is as follows:

    4: Required and full RT model capability
    2: Required and some RT model capability
    2: Somewhat required and full RT model capability
    1: Somewhat required and some RT model capability
    0: Not required 
    -1: Somewhat required and no RT model capability
    -4: Required and no RT model capability

In [7]:

#convert dataframe in categories to numerical values
cat_columns = cap.select_dtypes(['category']).columns
cap[cat_columns] = cap[cat_columns].apply(lambda x: x.cat.codes)

#convert dataframe in categories to numerical values
cat_columns = need.select_dtypes(['category']).columns
need[cat_columns] = need[cat_columns].apply(lambda x: x.cat.codes)

dummy=["test","test2"]
dummy.extend(need.iloc[:,0].values)
dummy.extend([0])
results = pd.DataFrame(dummy)                 #define dataframe with dummy values

for capcol in cap.columns: 
  for needcol in need.columns:
    this_need=(need.loc[:,needcol].values)     #requrirements vector for a specific usage
    this_cap=(cap.loc[:,capcol].values)         #capability vector for specific model
    res=this_need * this_cap                  
    reallybad=( (this_need**2)-this_cap == 4)  #make case where a need is required (2) but there is no capability
    res[reallybad]=-4                          #... and set that value to -4
    
    somewhatbad=( (this_need**2)-this_cap == 1)  #make case where a need is somewhat required (1) but there is no capability
    res[somewhatbad]=-1                          #... and set that value to -1

    su=sum(res)
    res_orig=res
    res=np.append(res,su)                    #calculate sum of results and append for TOTAL SCORE
    
    this=needcol+"_"+capcol                   #append dataframe with this individual capabilties + needs list
    this_res=[needcol,capcol]
    this_res.extend(res)
    results[this]=this_res

results=results.drop(columns=0)               #drop the dummy initial setup dataframe column
newindex=['Need','model']
newindex.extend(cap.index.values)
newindex.extend(["TOTAL SCORE"])

newindex
results["index"]=newindex
results=results.set_index("index")            #define and set the index values

## Plotting

This subroutine provides the means to control plotting color

In [8]:
def color_back(val):
    if val == 4:
      color = 'darkgreen'   #case for need and full capability
    else: 
      if val == 2:          #case for need and some capability, or some need and full capability
        color= 'limegreen'
      else:
        if val == 1:        #case for some need and some capability
          color='palegreen'
        else: 
          if val == -1:      #case for some need and no capability
            color='lightcoral'
          else:
            if val == -4:   #case for need and no capability
              color='red'
            else:
              color='white' #case for no need

    return 'background-color: %s' % color 

Here one can choose the specific RT "need" (by uncommenting the desired case). The results of that case are plotted below, and a .csv file with the results is saved to [path]/RTmodelsPACE_[case].csv .

In [12]:
#case = "Heritage"         #Heritage: the ability to (re)generate the same LUT's currently used in operational data processing
#case = "OperOciAc"        #Ability to generate LUT's for PACE/OCI required atmospheric correction
#case = "OperOciAer"       #Ability to generate LUT's for PACE/OCI required aerosol retrievals
#case = "OperOciCld"       #Ability to generate LUT's for PACE/OCI required cloud retrievals
#case = "OperOciLand"      #Ability to generate LUT's for PACE/OCI required land retrievals
#case = "EnhPolAerOc"      #Ability to support PACE enhanced (including polarimetry) aerosol/ocean retrievals
#case = "EnhPolAerLand"    #Ability to support PACE enhanced (including polarimetry) aerosol/land retrievals
#case = "EnhPolCld"     #Ability to support PACE enhanced (including polarimetry) cloud retrievals
#case = "PolOcean"         #Ability to support ocean property retrieval with PACE polarimeters
#case = "Explor"           #New PACE/OCI product investigation
case = "ExplorAirborne"   #Airborne polarimeter data exploration

new=results.loc[:,results.loc["Need"] == case]   #slice to the case selected above
newcolnames=new.loc["model"]
new=new.drop(['Need','model'])
new=new.rename(columns=newcolnames)              #update column names

outname=rt_path+'/RTmodelsPACE_'+case+'.csv'     #save csv file
new.to_csv(outname)

new_st=new.style.applymap(color_back)            #apply colors and plot
new_st



,AFRT,DAG,LINTRAN,GRASP SOS,UMBC,SORD,OSOAA,6SV,VLIDORT,DISORT,MYSTIC,AdSaa
index,,,,,,,,,,,,
Complete documentation,4,-4,2,2,4,2,4,4,2,4,4,2
Maintained version control,4,-4,4,4,4,-4,4,4,2,4,4,-4
Benchmarked,2,4,4,2,4,4,4,4,2,2,4,4
Intuitive inputs,2,2,2,-1,1,-1,2,1,1,2,2,1
Usable interface,4,2,2,4,4,-4,4,4,2,4,4,4
Spectral convolution,-4,-4,2,-4,4,-4,2,2,2,4,-4,-4
Modular components,4,4,4,2,4,4,-4,-4,2,2,2,2
Calculates Jacobians,-4,4,4,-4,-4,-4,-4,-4,2,-4,-4,-4
Vector,-4,4,4,4,4,4,4,4,4,-4,4,4
